Импортируем библиотеки

In [1]:
import csv
import time
import requests
import vk
import pandas as pd

Назначим глобальные переменные

In [9]:
token = 'Введите сюда свой token'
version = 5.131
count = 100

Напишем функцию парсинга постов на стене ('wall.get')

In [10]:
def take_all_posts():
    domain = '1tsprint'
    all_posts = []
    offset = 0
    while offset < 1000:
        response = requests.get('https://api.vk.com/method/wall.get',
                                params={
                                    'access_token': token,
                                    'v': version,
                                    'domain': domain,
                                    'count': count,
                                    'offset': offset
                                    }
                                )
        data = response.json()['response']['items']
        offset += 100
        all_posts.extend(data)
        time.sleep(0.5)
    return all_posts
all_posts = take_all_posts()

KeyError: 'response'

Объединим данные 'posts' и 'reposts' в общую таблицу и сохраним в файл *.csv

In [ ]:
list_posts, list_reposts = [], []
for post in all_posts:
    list_posts.append(post['text'])
    list_reposts.append(post['reposts']['count'])
count_posts_reposts = pd.concat([pd.Series(list_posts), pd.Series(list_reposts)], axis=1)
count_posts_reposts.index += 1
count_posts_reposts.columns = ['text_post', 'count_reposts']
count_posts_reposts.to_csv('count_posts_reposts.csv')
count_posts_reposts.head()

Напишем функцию нахождения количества подписчиков в группе

In [5]:
def count_followers():
    group_id = 215426617
    response = requests.get('https://api.vk.com/method/groups.getMembers',
                                params={
                                    'access_token': token,
                                    'v': version,
                                    'group_id': group_id
                                    }
                                )
    data = response.json()['response']['count']
    time.sleep(0.5)
    return data
count_followers = count_followers()
count_followers = pd.Series(count_followers)
count_followers

0    2502
dtype: int64

Посчитаем общее количество постов в группе

In [6]:
count_posts = pd.Series(count_posts_reposts['text_post'].count())
count_posts

0    322
dtype: int64

Посчитаем общее количество репостов к постам в группе

In [7]:
count_reposts = pd.Series(count_posts_reposts['count_reposts'].sum())
count_reposts

0    205
dtype: int64

Сведем все полученные данные в общую таблицу и сохраним в .csv файл

In [8]:
total_counts = pd.concat([count_posts, count_reposts, count_followers], axis=1)
total_counts.columns = ['count_posts', 'count_reposts', 'count_followers']
total_counts.to_csv('total_counts.csv')
total_counts

,count_posts,count_reposts,count_followers
0,322,205,2502
